In [2]:
import numpy as np
import pandas as pd
import random
import math
import statistics as stat
import matplotlib.pyplot as plt
import itertools
import warnings

In [3]:
def mmn_queueing_redundancy_dos_weibull(n,d,arrival_rate,scale,shape,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the redundancy method, the user needs to provide
    the number of servers (n), the number of copies to send (d), arrival rate (lambda) and scale/shape,
    in case of n >= 2, if scale/shape is provided as a scaler, then all servers
    share the same scale/shape, if scale/shape is provided as a vector, then all servers may have different scale/shape,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    Using the delete on start method.
    
    There are 6 inputs in this function,
    n:                the number of servers, a positive finite integer
    d:                the number of copies to send
    arrival_rate:     exponential rate of arrival
    scale:            weibull distribution scale
    shape:            weibull distribution shape
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)
        np.random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    if d < 1 or d > n:
        raise Exception("d needs to be in the range 1 <= d <= n")
    
    lamb =  arrival_rate
    
    if isinstance(scale, list):
        if len(scale) == n:
            scale = scale
        elif (n > 1 and len(scale) == 1):
            scale = n*scale
        else:
            raise Exception("Unmatched cardinality for scale and n")
    elif isinstance(scale, int) or isinstance(scale, float):
        if n == 1:
            scale = [scale]
        elif n > 1:
            scale = n*[scale]
    else:
        raise Exception("Please use int, float or list type for scale")
        
    if isinstance(shape, list):
        if len(shape) == n:
            shape = shape
        elif (n > 1 and len(shape) == 1):
            shape = n*shape
        else:
            raise Exception("Unmatched cardinality for shape and n")
    elif isinstance(shape, int) or isinstance(shape, float):
        if n == 1:
            shape = [shape]
        elif n > 1:
            shape = n*[shape]
    else:
        raise Exception("Please use int, float or list type for shape")
    
    # Simulating the Redundancy queues
    redun_server = np.zeros(n,dtype=int)
    for i in range(1,n+1):
        exec(f'redun_queue_{i} = []')
    copyhistory = pd.DataFrame(np.zeros(d,dtype=int)).T
    
    # Record the start/queue/finish timing for each arrival
    timing_table = np.zeros(5)
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,4+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1,dtype=int)  # keep track of the number customers
    no_completed_service = np.zeros(n+1,dtype=int)   # keep track of the number of completed services
    arrival_label = 0    # keep track on each arrival's label
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            copy_list = np.random.choice(np.arange(n),size=d,replace=False)
            arrival_label += 1
            
            timing_table = np.vstack([timing_table,np.zeros(5)])
            timing_table[-1,0] = arrival_label
            timing_table[-1,1] = event_calendar[-1,0]
            
            # If all servers are occupied
            if sum(redun_server[copy_list]==0) == 0:
                event_calendar[-1,queue_col[copy_list]] += 1
                event_calendar[-1,total_col[copy_list]] += 1
                for i in copy_list:
                    eval(f'redun_queue_{i+1}').append(arrival_label)
                new_history = pd.DataFrame(copy_list).T
                new_history.index = [f"{arrival_label}"]
                copyhistory = copyhistory.append(new_history)
            
            # If only one server is available
            elif sum(redun_server[copy_list]==0) == 1:
                avail_server = copy_list[np.where(redun_server[copy_list]==0)[0][0]]
                event_calendar[-1,table_col[avail_server]] += 1
                event_calendar[-1,avail_col[avail_server]] = event_calendar[-1,0] + \
                                                        scale[avail_server]*np.random.weibull(shape[avail_server])
                                                        
                event_calendar[-1,total_col[avail_server]] += 1
                redun_server[avail_server] = arrival_label
                no_customers[avail_server] += 1
                
                timing_table[-1,2] = event_calendar[-1,0]
                timing_table[-1,4] = avail_server+1
                
            # If more than one servers are available
            else:
                avail_server = random.choice(copy_list[np.where(redun_server[copy_list]==0)[0]])
                event_calendar[-1,table_col[avail_server]] += 1
                event_calendar[-1,avail_col[avail_server]] = event_calendar[-1,0] + \
                                                        scale[avail_server]*np.random.weibull(shape[avail_server])
                event_calendar[-1,total_col[avail_server]] += 1
                redun_server[avail_server] = arrival_label
                no_customers[avail_server] += 1
                
                timing_table[-1,2] = event_calendar[-1,0]
                timing_table[-1,4] = avail_server+1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            event_calendar[-1,-1] += 1
            
            timing_table[redun_server[server_completion],3] = event_calendar[-1,0]
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = 0
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        scale[server_completion]*np.random.weibull(shape[server_completion])
                redun_server[server_completion] = eval(f'redun_queue_{server_completion+1}')[0]
                for i in copyhistory.loc[f"{redun_server[server_completion]}"]:
                    event_calendar[-1,queue_col[i]] -= 1
                    event_calendar[-1,total_col[i]] -= 1
                    eval(f'redun_queue_{i+1}').remove(redun_server[server_completion])
                copyhistory = copyhistory.drop(f"{redun_server[server_completion]}")
                no_customers[server_completion] += 1
                
                timing_table[redun_server[server_completion],2] = event_calendar[-1,0]
                timing_table[redun_server[server_completion],4] = server_completion+1
                
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)])) + \
                    ['Live_track']
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    timing_table = pd.DataFrame(timing_table)
    timing_table.columns = ['Arrival_label','Start','Queue','Finish','Server']
    timing_table = timing_table[timing_table['Finish'] != 0]
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time
    awt_data = np.array([])
    for i in range(1,n+1):
        awt_table = timing_table[timing_table['Server']==i]
        if awt_table.shape[0] != 0:
            awt_data = np.append(awt_data,stat.mean(awt_table['Queue']-awt_table['Start']))
        else:
            awt_data = np.append(awt_data,0)
    awt_data = np.append(awt_data,stat.mean(timing_table['Queue']-timing_table['Start']))
    awt = pd.DataFrame(awt_data.round(3)).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time
    act_data = np.array([])
    for i in range(1,n+1):
        act_table = timing_table[timing_table['Server']==i]
        if act_table.shape[0] != 0:
            act_data = np.append(act_data,stat.mean(act_table['Finish']-act_table['Start']))
        else:
            act_data = np.append(act_data,0)
    act_data = np.append(act_data,stat.mean(timing_table['Finish']-timing_table['Start']))
    act = pd.DataFrame(act_data.round(3)).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]

    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "scale": scale,
        "shape": shape,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat, timing_table


In [4]:
def mmn_queueing_redundancy_doc_weibull(n,d,arrival_rate,scale,shape,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the redundancy method, the user needs to provide
    the number of servers (n), the number of copies to send (d), arrival rate (lambda) and scale/shape,
    in case of n >= 2, if scale/shape is provided as a scaler, then all servers
    share the same scale/shape, if scale/shape is provided as a vector, then all servers may have different scale/shape,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    Using the delete on completion method.
    
    There are 6 inputs in this function,
    n:                the number of servers, a positive finite integer
    d:                the number of copies to send
    arrival_rate:     exponential rate of arrival
    service_rate:     exponential rate of service time
    simulation_time:  the simulation time, a positive finite number
    simuseed:         the simulation seed number
    
    
    There are 8 outputs in this function,
    event_calendar:   the event calendar for the whole simulation, containing all important
                      information needed (current time, next customer arrival time, finish time,
                      server occupancy variable, server next available time, server queue length,
                      server total, grand queue length and grand total)
    utiliz:           the utilization for each server
    ans:              the average number in the system for each server and grand average
    aql:              the average queue length for each server and grand average
    awt:              the average waiting time for each customer and grand average
    act:              the average cycle time for each customer and grand average
    no_cust_arr_comp: the table illustrating the number of arrived customers and the
                      the number of completed services for each server and grand total
    other_stat:       return other calculated statistics, for plotting purposes
    
    Code writer: Yaozu Li
    uoe S.No: 2298215
    
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)
        np.random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    if d < 1 or d > n:
        raise Exception("d needs to be in the range 1 <= d <= n")
    
    lamb =  arrival_rate
    
    if isinstance(scale, list):
        if len(scale) == n:
            scale = scale
        elif (n > 1 and len(scale) == 1):
            scale = n*scale
        else:
            raise Exception("Unmatched cardinality for scale and n")
    elif isinstance(scale, int) or isinstance(scale, float):
        if n == 1:
            scale = [scale]
        elif n > 1:
            scale = n*[scale]
    else:
        raise Exception("Please use int, float or list type for scale")
        
    if isinstance(shape, list):
        if len(shape) == n:
            shape = shape
        elif (n > 1 and len(shape) == 1):
            shape = n*shape
        else:
            raise Exception("Unmatched cardinality for shape and n")
    elif isinstance(shape, int) or isinstance(shape, float):
        if n == 1:
            shape = [shape]
        elif n > 1:
            shape = n*[shape]
    else:
        raise Exception("Please use int, float or list type for shape")
    
    # Simulating the Redundancy queues
    redun_server = np.zeros(n,dtype=int)
    for i in range(1,n+1):
        exec(f'redun_queue_{i} = []')
    copyhistory = pd.DataFrame(np.zeros(d,dtype=int)).T
    
    # Record the start/queue/finish timing for each arrival
    timing_table = np.zeros(5)
    queue_record = np.zeros(n)
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,4+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    no_customers = np.zeros(n+1,dtype=int)  # keep track of the number customers
    no_completed_service = np.zeros(n+1,dtype=int)   # keep track of the number of completed services
    arrival_label = 0    # keep track on each arrival's label
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            copy_list = np.random.choice(np.arange(n),size=d,replace=False)
            arrival_label += 1
            new_history = pd.DataFrame(copy_list).T
            new_history.index = [f"{arrival_label}"]
            copyhistory = copyhistory.append(new_history)
            
            timing_table = np.vstack([timing_table,np.zeros(5)])
            timing_table[-1,0] = arrival_label
            timing_table[-1,1] = event_calendar[-1,0]
            queue_record = np.vstack([queue_record,np.zeros(n)])
            
            for i in copy_list:
                no_customers[i] += 1
                
                # If the server is idle
                if event_calendar[-1,table_col[i]] == 0:
                    event_calendar[-1,table_col[i]] += 1
                    event_calendar[-1,avail_col[i]] = event_calendar[-1,0] + \
                                                      scale[i]*np.random.weibull(shape[i])
                    event_calendar[-1,total_col[i]] += 1
                    redun_server[i] = arrival_label
                    
                    queue_record[arrival_label,i] = event_calendar[-1,0]
                    
                # If the server is occupied
                elif event_calendar[-1,table_col[i]] == 1:
                    event_calendar[-1,queue_col[i]] += 1
                    event_calendar[-1,total_col[i]] += 1
                    eval(f'redun_queue_{i+1}').append(arrival_label)
        
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            no_completed_service[server_completion] += 1
            job_completion = redun_server[server_completion]
            event_calendar[-1,-1] += 1
            
            timing_table[job_completion,2] = queue_record[job_completion,server_completion]
            timing_table[job_completion,3] = event_calendar[-1,0]
            timing_table[job_completion,4] = server_completion+1
            
            # For the completion server
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = 0
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                    scale[server_completion]*np.random.weibull(shape[server_completion])
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
                redun_server[server_completion] = eval(f'redun_queue_{server_completion+1}')[0]
                eval(f'redun_queue_{server_completion+1}').remove(redun_server[server_completion])
            
                queue_record[redun_server[server_completion],server_completion] = event_calendar[-1,0]
            
            # For the other servers in the copylist
            for i in copyhistory.loc[f"{job_completion}"]:
                if i != server_completion:
                    no_customers[i] -= 1
                
                    # If the completed job is undertaken
                    if redun_server[i] == job_completion:
                        
                        # If there is no queue behind
                        if event_calendar[-1,queue_col[i]] == 0:
                            event_calendar[-1,table_col[i]] -= 1
                            event_calendar[-1,avail_col[i]] = math.nan
                            event_calendar[-1,total_col[i]] -= 1
                            redun_server[i] = 0

                        # Else if there is a queue behind
                        elif event_calendar[-1,queue_col[i]] > 0:
                            event_calendar[-1,avail_col[i]] = event_calendar[-1,0] + \
                                                                    scale[i]*np.random.weibull(shape[i])
                            event_calendar[-1,queue_col[i]] -= 1
                            event_calendar[-1,total_col[i]] -= 1
                            redun_server[i] = eval(f'redun_queue_{i+1}')[0]
                            eval(f'redun_queue_{i+1}').remove(redun_server[i])
                            
                            queue_record[redun_server[i],i] = event_calendar[-1,0]
                    
                    # Else if the completed job is not undertaken
                    elif redun_server[i] != job_completion:

                        event_calendar[-1,queue_col[i]] -= 1
                        event_calendar[-1,total_col[i]] -= 1
                        eval(f'redun_queue_{i+1}').remove(job_completion)

            copyhistory = copyhistory.drop(f"{job_completion}")


                
                
    no_customers[-1] = sum(no_customers[:-1])
    no_completed_service[-1] = sum(no_completed_service[:-1])
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)])) + \
                    ['Live_track']
    event_calendar["Grand Queue"] = event_calendar.iloc[:,queue_col].sum(axis=1)
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    timing_table = pd.DataFrame(timing_table)
    timing_table.columns = ['Arrival_label','Start','Queue','Finish','Server']
    timing_table = timing_table[timing_table['Finish'] != 0]
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average number in the system" for i in range(1,n+1)]+["Grand Average number in the system"]
    
    # Calculating the average queue length
    aql_server = event_calendar.iloc[:-1,np.append(queue_col,event_calendar.shape[1]-2)].to_numpy()
    aql_data = ((time3 @ aql_server)/simulation_time).round(3)
    aql = pd.DataFrame(aql_data).transpose()
    aql.columns = [f"Server{i} Average queue length" for i in range(1,n+1)]+["Grand Average queue length"]
    
    # Average waiting time
    awt_data = np.array([])
    for i in range(1,n+1):
        awt_table = timing_table[timing_table['Server']==i]
        if awt_table.shape[0] != 0:
            awt_data = np.append(awt_data,stat.mean(awt_table['Queue']-awt_table['Start']))
        else:
            awt_data = np.append(awt_data,0)
    awt_data = np.append(awt_data,stat.mean(timing_table['Queue']-timing_table['Start']))
    awt = pd.DataFrame(awt_data.round(3)).transpose()
    awt.columns = [f"Server{i} Average waiting time" for i in range(1,n+1)]+["Grand Average waiting time"]
    
    # Average cycle time
    act_data = np.array([])
    for i in range(1,n+1):
        act_table = timing_table[timing_table['Server']==i]
        if act_table.shape[0] != 0:
            act_data = np.append(act_data,stat.mean(act_table['Finish']-act_table['Start']))
        else:
            act_data = np.append(act_data,0)
    act_data = np.append(act_data,stat.mean(timing_table['Finish']-timing_table['Start']))
    act = pd.DataFrame(act_data.round(3)).transpose()
    act.columns = [f"Server{i} Average cycle time" for i in range(1,n+1)]+["Grand Average cycle time"]

    # Show the number of customers arrived and completed
    no_cust_arr_comp = pd.DataFrame(np.vstack([no_customers,no_completed_service]))
    no_cust_arr_comp.columns = [f"Server{i}" for i in range(1,n+1)]+["Grand total"]
    no_cust_arr_comp.index = ["Arrived customers", "Completed services"]
    
    # Find the maximum queue length
    mql = event_calendar.iloc[:,queue_col].max()
    
    # For other statistics
    other_stat = {
        "n": n,
        "lamb": lamb,
        "scale": scale,
        "shape": shape,
        "simulation_time": simulation_time,
        "table_col": table_col,
        "avail_col": avail_col,
        "queue_col": queue_col,
        "total_col": total_col,
        "compare_col": compare_col,
        "time3": time3,
        "mql": mql
    }
    
    # Return the event calendar and all performance measures table
    return event_calendar, utiliz, ans, aql, awt, act, no_cust_arr_comp, other_stat, timing_table


In [ ]:
mmn_queueing_redundancy_doc_weibull(n,d,arrival_rate,scale,shape,simulation_time=60,simuseed=math.nan)

In [5]:
aaa = mmn_queueing_redundancy_dos_weibull(5,5,20,scale=10,shape=1,simulation_time=20,simuseed=8 )
bbb = mmn_queueing_redundancy_doc_weibull(5,5,20,scale=10,shape=1,simulation_time=20,simuseed=8)

In [42]:
aaa[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization,Server5 Utilization
0,0.991,0.988,0.999,0.99,0.986


In [43]:
bbb[1]

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization,Server5 Utilization
0,0.995,0.995,0.995,0.995,0.995


In [44]:
aaa[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Server5 Average number in the system,Grand Average number in the system
0,186.322,186.319,186.33,186.321,186.317,931.61


In [47]:
bbb[2]

,Server1 Average number in the system,Server2 Average number in the system,Server3 Average number in the system,Server4 Average number in the system,Server5 Average number in the system,Grand Average number in the system
0,2.117,1.916,2.182,1.953,2.183,10.352


In [48]:
aaa[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Server5 Average queue length,Grand Average queue length
0,0.919,0.732,0.802,0.836,0.987,4.277


In [49]:
bbb[3]

,Server1 Average queue length,Server2 Average queue length,Server3 Average queue length,Server4 Average queue length,Server5 Average queue length,Grand Average queue length
0,1.303,1.147,1.369,1.214,1.372,6.405


In [49]:
aaa[4]

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Server5 Average waiting time,Grand Average waiting time
0,0.0,1.151,0.0,0.0,0.0,0.691


In [51]:
bbb[4]

,Server1 Average waiting time,Server2 Average waiting time,Server3 Average waiting time,Server4 Average waiting time,Server5 Average waiting time,Grand Average waiting time
0,0.383,0.207,0.856,0.213,0.312,0.31


In [53]:
aaa[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Server5 Average cycle time,Grand Average cycle time
0,0.546,0.283,0.867,0.272,0.49,0.424


In [52]:
bbb[5]

,Server1 Average cycle time,Server2 Average cycle time,Server3 Average cycle time,Server4 Average cycle time,Server5 Average cycle time,Grand Average cycle time
0,0.623,0.345,1.364,0.343,0.496,0.501


In [50]:
aaa[6]

,Server1,Server2,Server3,Server4,Server5,Grand total
Arrived customers,2,4,1,2,1,10
Completed services,1,3,0,1,0,5


In [51]:
bbb[6]

,Server1,Server2,Server3,Server4,Server5,Grand total
Arrived customers,372,371,373,371,370,1857
Completed services,2,1,3,1,0,7


In [6]:
aaa[0]

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,...,Server4 Available Time,Server4 Queue,Server4 Total,Server5,Server5 Available Time,Server5 Queue,Server5 Total,Live_track,Grand Queue,Grand Total
1,0.000000,0.012855,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,0.012855,0.176753,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0
3,0.176753,0.187482,20.0,0.0,NaN,0.0,0.0,1.0,0.291396,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,2.0
4,0.187482,0.191933,20.0,1.0,6.695867,0.0,1.0,1.0,0.291396,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,3.0
5,0.191933,0.275304,20.0,1.0,6.695867,0.0,1.0,1.0,0.291396,0.0,...,5.344912,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,19.789811,19.843445,20.0,1.0,51.979899,407.0,408.0,1.0,33.535483,407.0,...,57.426805,407.0,408.0,1.0,21.055001,407.0,408.0,5.0,2035.0,2040.0
424,19.843445,19.918817,20.0,1.0,51.979899,408.0,409.0,1.0,33.535483,408.0,...,57.426805,408.0,409.0,1.0,21.055001,408.0,409.0,5.0,2040.0,2045.0
425,19.918817,19.996078,20.0,1.0,51.979899,409.0,410.0,1.0,33.535483,409.0,...,57.426805,409.0,410.0,1.0,21.055001,409.0,410.0,5.0,2045.0,2050.0
426,19.996078,20.057140,20.0,1.0,51.979899,410.0,411.0,1.0,33.535483,410.0,...,57.426805,410.0,411.0,1.0,21.055001,410.0,411.0,5.0,2050.0,2055.0


In [7]:
bbb[0]

,Time,Next Customer,Finish Time,Server1,Server1 Available Time,Server1 Queue,Server1 Total,Server2,Server2 Available Time,Server2 Queue,...,Server4 Available Time,Server4 Queue,Server4 Total,Server5,Server5 Available Time,Server5 Queue,Server5 Total,Live_track,Grand Queue,Grand Total
1,0.000000,0.012855,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,0.012855,0.176753,20.0,1.0,0.127497,0.0,1.0,1.0,2.661998,0.0,...,5.642272,0.0,1.0,1.0,20.353296,0.0,1.0,0.0,0.0,5.0
3,0.127497,0.176753,20.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0
4,0.176753,0.183506,20.0,1.0,6.685138,0.0,1.0,1.0,12.637715,0.0,...,14.485302,0.0,1.0,1.0,8.281577,0.0,1.0,1.0,0.0,5.0
5,0.183506,0.244514,20.0,1.0,6.685138,1.0,2.0,1.0,12.637715,1.0,...,14.485302,1.0,2.0,1.0,8.281577,1.0,2.0,1.0,5.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,19.885469,19.907890,20.0,1.0,29.945328,366.0,367.0,1.0,27.847578,366.0,...,30.620464,366.0,367.0,1.0,42.321373,366.0,367.0,7.0,1830.0,1835.0
383,19.907890,19.953762,20.0,1.0,29.945328,367.0,368.0,1.0,27.847578,367.0,...,30.620464,367.0,368.0,1.0,42.321373,367.0,368.0,7.0,1835.0,1840.0
384,19.953762,19.984243,20.0,1.0,29.945328,368.0,369.0,1.0,27.847578,368.0,...,30.620464,368.0,369.0,1.0,42.321373,368.0,369.0,7.0,1840.0,1845.0
385,19.984243,20.015989,20.0,1.0,29.945328,369.0,370.0,1.0,27.847578,369.0,...,30.620464,369.0,370.0,1.0,42.321373,369.0,370.0,7.0,1845.0,1850.0
